In [0]:
%sql
SHOW TABLES;

database,tableName,isTemporary
default,movies,false
default,orders,false


In [0]:
%python
events = spark.table("default.orders")


In [0]:
events.printSchema()
events.show(5)


root
 |-- order_date: date (nullable = true)
 |-- country: string (nullable = true)
 |-- order_id: long (nullable = true)
 |-- product: string (nullable = true)
 |-- qty: long (nullable = true)
 |-- price: double (nullable = true)

+----------+-------+--------+--------+---+-----+
|order_date|country|order_id| product|qty|price|
+----------+-------+--------+--------+---+-----+
|2024-02-16|     IN|    1000|   Shoes|  2|54.37|
|2024-02-01|     CA|    1001|Backpack|  4|77.01|
|2024-02-03|     AU|    1002|  Jacket|  1|95.07|
|2024-03-01|     UK|    1003|   Jeans|  1|42.03|
|2024-01-31|     IN|    1004| T-Shirt|  3|15.94|
+----------+-------+--------+--------+---+-----+
only showing top 5 rows


In [0]:
events.select("order_id", "product", "qty", "price").show(10)


+--------+--------+---+------+
|order_id| product|qty| price|
+--------+--------+---+------+
|    1000|   Shoes|  2| 54.37|
|    1001|Backpack|  4| 77.01|
|    1002|  Jacket|  1| 95.07|
|    1003|   Jeans|  1| 42.03|
|    1004| T-Shirt|  3| 15.94|
|    1005|   Watch|  2|131.49|
|    1006|   Shoes|  3| 50.83|
|    1007|Backpack|  3| 85.85|
|    1008| T-Shirt|  2| 16.94|
|    1009|   Jeans|  4| 37.45|
+--------+--------+---+------+
only showing top 10 rows


In [0]:
events.filter("price > 50").show(10)
events.filter("price > 50").count()

+----------+-------+--------+--------+---+------+
|order_date|country|order_id| product|qty| price|
+----------+-------+--------+--------+---+------+
|2024-02-16|     IN|    1000|   Shoes|  2| 54.37|
|2024-02-01|     CA|    1001|Backpack|  4| 77.01|
|2024-02-03|     AU|    1002|  Jacket|  1| 95.07|
|2024-01-17|     IN|    1005|   Watch|  2|131.49|
|2024-01-15|     UK|    1006|   Shoes|  3| 50.83|
|2024-01-18|     IN|    1007|Backpack|  3| 85.85|
|2024-02-11|     US|    1015|  Jacket|  5| 84.32|
|2024-01-04|     AU|    1016|Backpack|  3| 85.08|
|2024-02-25|     US|    1017|Backpack|  5| 82.64|
|2024-01-06|     AU|    1018|   Shoes|  5| 53.75|
+----------+-------+--------+--------+---+------+
only showing top 10 rows


47

In [0]:
from pyspark.sql.functions import col

events2 = events.withColumn("total_amount", col("qty") * col("price"))
events2.show(5)


+----------+-------+--------+--------+---+-----+------------+
|order_date|country|order_id| product|qty|price|total_amount|
+----------+-------+--------+--------+---+-----+------------+
|2024-02-16|     IN|    1000|   Shoes|  2|54.37|      108.74|
|2024-02-01|     CA|    1001|Backpack|  4|77.01|      308.04|
|2024-02-03|     AU|    1002|  Jacket|  1|95.07|       95.07|
|2024-03-01|     UK|    1003|   Jeans|  1|42.03|       42.03|
|2024-01-31|     IN|    1004| T-Shirt|  3|15.94|       47.82|
+----------+-------+--------+--------+---+-----+------------+
only showing top 5 rows


In [0]:
from pyspark.sql.functions import sum

sales_by_country = (
    events2.groupBy("country")
    .agg(sum("total_amount").alias("total_sales"))
    .orderBy(col("total_sales").desc())
)

sales_by_country.show()


+-------+------------------+
|country|       total_sales|
+-------+------------------+
|     US|           4255.98|
|     CA| 4176.000000000001|
|     UK|3678.6700000000005|
|     AU|           2477.86|
|     IN|2000.8799999999999|
+-------+------------------+



In [0]:
top_products = (
    events2.groupBy("product")
    .agg(sum("total_amount").alias("revenue"))
    .orderBy(col("revenue").desc())
    .limit(5)
)

top_products.show()


+--------+------------------+
| product|           revenue|
+--------+------------------+
|   Watch|           4733.32|
|Backpack|           3627.14|
|  Jacket|           2886.27|
|   Shoes|           2776.89|
|   Jeans|1616.2599999999998|
+--------+------------------+



In [0]:
## %sql magic command 

In [0]:
events2.createOrReplaceTempView("orders")



In [0]:
%sql
SELECT country, COUNT(*) AS total_orders
FROM orders
GROUP BY country
ORDER BY total_orders DESC;


country,total_orders
CA,24
US,23
UK,19
IN,17
AU,17


In [0]:
%sql
SELECT product, ROUND(SUM(qty * price), 2) AS revenue
FROM orders
GROUP BY product
ORDER BY revenue DESC
LIMIT 5;


product,revenue
Watch,4733.32
Backpack,3627.14
Jacket,2886.27
Shoes,2776.89
Jeans,1616.26


In [0]:
events = spark.table("default.orders")
events.show(5)
from pyspark.sql.functions import col

events2 = events.withColumn("total_amount", col("qty") * col("price"))
events2.show(5)

from pyspark.sql.functions import sum

top_products = (
    events2.groupBy("product")
    .agg(sum("total_amount").alias("revenue"))
    .orderBy(col("revenue").desc())
    .limit(5)
)

top_products.show()




+----------+-------+--------+--------+---+-----+
|order_date|country|order_id| product|qty|price|
+----------+-------+--------+--------+---+-----+
|2024-02-16|     IN|    1000|   Shoes|  2|54.37|
|2024-02-01|     CA|    1001|Backpack|  4|77.01|
|2024-02-03|     AU|    1002|  Jacket|  1|95.07|
|2024-03-01|     UK|    1003|   Jeans|  1|42.03|
|2024-01-31|     IN|    1004| T-Shirt|  3|15.94|
+----------+-------+--------+--------+---+-----+
only showing top 5 rows
+----------+-------+--------+--------+---+-----+------------+
|order_date|country|order_id| product|qty|price|total_amount|
+----------+-------+--------+--------+---+-----+------------+
|2024-02-16|     IN|    1000|   Shoes|  2|54.37|      108.74|
|2024-02-01|     CA|    1001|Backpack|  4|77.01|      308.04|
|2024-02-03|     AU|    1002|  Jacket|  1|95.07|       95.07|
|2024-03-01|     UK|    1003|   Jeans|  1|42.03|       42.03|
|2024-01-31|     IN|    1004| T-Shirt|  3|15.94|       47.82|
+----------+-------+--------+--------+-

In [0]:
top_products.write.mode("overwrite").saveAsTable("default.day2_top_products")


In [0]:
%sql
SELECT * FROM default.day2_top_products;


product,revenue
Watch,4733.32
Backpack,3627.14
Jacket,2886.27
Shoes,2776.89
Jeans,1616.2599999999998


In [0]:
top_products.write.mode("overwrite").saveAsTable("default.day2_top_products")


In [0]:
%sql
SELECT * FROM default.day2_top_products;


product,revenue
Watch,4733.32
Backpack,3627.14
Jacket,2886.27
Shoes,2776.89
Jeans,1616.2599999999998
